In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import torch
import math
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torch.nn as nn
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import pickle

mycase="tw"
casenum="6"


all_path="/content/drive/MyDrive/Colab_Notebooks/2023_MEP/1107_data/"

path_s=all_path+mycase+"_s_"+casenum+".pkl"
path_path=all_path+mycase+"_path_"+casenum+".pkl"
path_cos=all_path+mycase+"_cos_"+casenum+".pkl"
path_energy=all_path+mycase+"_energy_"+casenum+".pkl"
path_force=all_path+mycase+"_force_"+casenum+".pkl"
path_cosloss=all_path+mycase+"_coslosss_"+casenum+".pkl"
path_lmax=all_path+mycase+"_lmax_"+casenum+".pkl"
path_lg=all_path+mycase+"_lg_"+casenum+".pkl"

plot_use_all=[]

torch.set_printoptions(precision=10)

model_Parameters_name = all_path+"model_MEP_"+mycase[0:2]+"_"+casenum+"pretrain"+".pth" # Path
model_name= all_path+"model_MEP_"+mycase+"_"+casenum+".pth"



load_model = True
batches = 2000
beta = 10
learning_rate = 1e-4
dimension = 2

x_lb=-2
x_ub=2
y_lb=-1.5
y_ub=2.5


alpha1=1
alpha2=10
alpha3=1


def V_tensor(x):
  x_all=torch.cat(x,axis=1)
  Z=3*torch.exp(-x_all[:,0]**2-(x_all[:,1]-1/3)**2)-3*torch.exp(-x_all[:,0]**2-(x_all[:,1]-5/3)**2)-5*torch.exp(-(x_all[:,0]-1)**2-x_all[:,1]**2)-5*torch.exp(-(x_all[:,0]+1)**2-x_all[:,1]**2)#+1/5*x_all[:,0]**4+1/5*(x_all[:,1]-1/3)**4
  return Z


def V_tensor_grad(x): # first order derivative of V w.r.t x
    grad_Vx=[]
    output = V_tensor(x)
    for i in range(dimension):
      grad_Vx.append(torch.autograd.grad(outputs=output, inputs=x[i], grad_outputs=torch.ones_like(output), create_graph=True)[0])

    return grad_Vx

def fig_cos_V_force(s,cos2,g,x_pred_list):
    fig = plt.figure(figsize=(15,4))
    fig.subplots_adjust(hspace=0.4, wspace=0.4)


    plt.subplot(1,3,1)
    plt.plot(s,cos2,'b.')
    plt.ylim(-0.1,1.1)
    plt.title("$Cos^2$")

    plt.subplot(1,3,2)
    plt.plot(s,V_tensor(x_pred_list).cpu().detach().numpy(), 'b.')
    plt.title("$Energy$")

    plt.subplot(1,3,3)

    force=np.sqrt(np.sum(g*g, axis=1))
    plt.plot(s,force, 'b.')
    plt.title("$Force$")

def fig_loss_batch(plt_batch,loss_batch):
    plt.figure(figsize=(5,4))
    plt.plot(plt_batch,loss_batch,'b-')
    plt.xlabel("Batches")
    plt.ylabel("Loss")

def fig_cos(plt_batch,cos_batch,lmax_batch,lg_batch):
    fig = plt.figure(figsize=(15,4))
    fig.subplots_adjust(hspace=0.4, wspace=0.4)

    plt.subplot(1,3,1)
    plt.semilogy(plt_batch,cos_batch,'b-')
    plt.xlabel("Batches")
    plt.ylabel("$\int 1-Cos^2(force,tangent) ds$")

    plt.subplot(1,3,2)
    plt.plot(plt_batch,lmax_batch,'b-')
    plt.xlabel("Batches")
    plt.ylabel("$lmax$")

    plt.subplot(1,3,3)
    plt.plot(plt_batch,lg_batch,'b-')
    plt.xlabel("Batches")
    plt.ylabel("$l_g$")

def fig_countour(x_pred):
    plt.figure(figsize=(5,4))
    x = np.linspace(x_lb, x_ub,num=51,endpoint=True)
    y = np.linspace(y_lb, y_ub,num=51,endpoint=True)

    X,Y = np.meshgrid(x,y)
    X_new=X.reshape(-1,1)
    Y_new=Y.reshape(-1,1)
    XY=np.hstack((X_new,Y_new))
    for i in range(dimension-2):
      XY=np.hstack((XY,np.zeros(X_new.shape)))

    XY_tensor=torch.from_numpy(XY)
    X_list = []
    for i in range(dimension):
      X_list.append(XY_tensor[:, i:i+1].to(device))
    Z = V_tensor(X_list)

    Z_new=Z.reshape(X.shape).cpu().detach().numpy()


    plt.figure()
    CS = plt.contourf(X,Y,Z_new,50,cmap=mpl.cm.jet)
    plt.colorbar(CS)
    x_plot=x_pred.cpu().detach().numpy()
    plt.plot(x_plot[:,0],x_plot[:,1],"r.",markersize=5)
    plt.xlim((x_lb,x_ub))
    plt.ylim((y_lb,y_ub))
    plt.xlabel('$x_1$')
    plt.ylabel('$x_2$')


class NeuralNetwork(nn.Module):
    def __init__(self,st,ed):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_tanh_stack = nn.Sequential(
            nn.Linear(1,16),
            nn.Sigmoid(),
            nn.Linear(16, 32),
            nn.Sigmoid(),
            nn.Linear(32, 32),
            nn.Sigmoid(),
            nn.Linear(32, 16),
            nn.Sigmoid(),
            nn.Linear(16, dimension)
        )
        self.startpoint=Variable(torch.from_numpy(np.array([st])),requires_grad=False).to(device)
        self.endpoint=Variable(torch.from_numpy(np.array([ed])),requires_grad=False).to(device)


    def forward(self, s):
        s = self.flatten(s)
        x_pred = self.linear_tanh_stack(s)
        out=s*(1-s)*x_pred + (1-s)*self.startpoint + s*self.endpoint
        return out



def train(model):
    global my_V
    cos_batch=[]
    loss_batch=[]
    plt_batch=[]

    lg_batch=[]
    lmax_batch=[]
    if load_model:
        model.load_state_dict(torch.load(model_Parameters_name))



    for i in range(batches):
        if i<4000:
          alpha1=0.1
          alpha2=0.1
          alpha3=0.0000001
        else:
          alpha1=0.0001
          alpha2=0.1
          alpha3=0.0000001
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        s = np.random.uniform(0.001,0.999,(500,1))
        s = Variable(torch.from_numpy(s),requires_grad=True).to(device)
        x_pred = model(s)
        x_pred_list = []
        for n in range(dimension):
            x_pred_list.append(x_pred[:, n:n+1])

        v = V_tensor(x_pred_list).to(device)
        gradx_list=[]
        gradgx_list=[]
        for n in range(dimension):
            gradx_list.append(torch.autograd.grad(outputs=x_pred_list[n],inputs=s,grad_outputs=torch.ones_like(s),create_graph=True)[0]) # First order derivative of x w.r.t s
            gradgx_list.append(torch.autograd.grad(outputs=gradx_list[n],inputs=s,grad_outputs=torch.ones_like(s),create_graph=True)[0]) # Second order derivative of x w.r.t s

        partial_s_x = torch.cat(gradx_list, axis=1)
        dot_partial_s_x = torch.sqrt(torch.sum(partial_s_x*partial_s_x, axis=1, keepdim=False)) #s对神经网络的一阶导数的数量积

        loss_1=1/beta*torch.log(torch.mean(torch.exp(v*beta)*dot_partial_s_x))

        g_all = V_tensor_grad(x_pred_list)
        g=torch.cat(g_all, axis=1)
        cos2 = (torch.sum(partial_s_x*g,axis=1, keepdim=True)/(torch.sum(g*g, axis=1, keepdim=True)**0.5*torch.sum(partial_s_x*partial_s_x, axis=1, keepdim=True)**0.5))**2
        loss_2 = torch.mean(1-cos2)

        cons=0
        for n in range(dimension):
            cons += torch.sum(gradx_list[n]*gradgx_list[n], axis=1, keepdim=True)
        loss_3 = torch.mean(cons**2)

        loss = alpha1*loss_1 +alpha2*loss_2+alpha3*loss_3

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i) % 50 == 0:
            ss = np.linspace(0.001,0.999,1000,endpoint=True)

            ss = ss.reshape(-1,1)
            ss = Variable(torch.from_numpy(ss),requires_grad=True).to(device)
            xx = model(ss)
            gradx_list_plot=[]
            x_pred_list_plot = []
            for n in range(dimension):
                x_pred_list_plot.append(xx[:, n:n+1])
            for n in range(dimension):
                gradx_list_plot.append(torch.autograd.grad(outputs=x_pred_list_plot[n],inputs=ss,grad_outputs=torch.ones_like(ss),create_graph=True)[0]) # First order derivative of x w.r.t s
            partial_s_x_plot = torch.cat(gradx_list_plot, axis=1)

            g_all_plot = V_tensor_grad(x_pred_list_plot)
            g_plot=torch.cat(g_all_plot, axis=1)
            cos2_plot = (torch.sum(partial_s_x_plot*g_plot,axis=1, keepdim=True)/(torch.sum(g_plot*g_plot, axis=1, keepdim=True)**0.5*torch.sum(partial_s_x_plot*partial_s_x_plot, axis=1, keepdim=True)**0.5))**2
            cos_plot_loss = torch.mean(1-cos2_plot)
            cos_batch.append(cos_plot_loss.item())

            U_max=torch.max(V_tensor(x_pred_list_plot))
            lmax_batch.append(U_max.item())

            dot_partial_s_x_plot = torch.sqrt(torch.sum(partial_s_x_plot*partial_s_x_plot, axis=1, keepdim=False))
            dot_g_plot= torch.sqrt(torch.sum(g_plot*g_plot, axis=1, keepdim=False))
            lg=torch.mean(dot_g_plot*dot_partial_s_x_plot)

            lg_batch.append(lg.item())

            loss, batch = alpha1*loss_1.item() + alpha2*loss_2.item() + alpha3*loss_3.item(), i
            print(f'batches: {batch+1}')
            print(f'loss1: {alpha1*loss_1} loss2: {alpha2*loss_2} loss3: {alpha3*loss_3}')

            loss_batch.append(loss)
            plt_batch.append(i+1)
            plt.figure()

        if (i+1) % 500 == 0:
            print("cos_plot_loss.item()=",cos_plot_loss.item())
            fig_cos(plt_batch,cos_batch,lmax_batch,lg_batch)
            fig_loss_batch(plt_batch,loss_batch)
            fig_cos_V_force(s.cpu().detach().numpy(),cos2.cpu().detach().numpy(),g.cpu().detach().numpy(),x_pred_list)  # Why [1:-1
            fig_countour(x_pred)
            plt.show()


            torch.save(model.state_dict(), model_name)
            print("Saved PyTorch Model State to " +str(model_name))

    my_V=V_tensor(x_pred_list_plot)

    output = open(path_s, 'wb')
    pickle.dump(s.cpu().detach().numpy(),output)
    output.close()

    output = open(path_path, 'wb')
    pickle.dump(x_pred.cpu().detach().numpy(),output)
    output.close()

    output = open(path_cos, 'wb')
    pickle.dump(cos2.cpu().detach().numpy(),output)
    output.close()

    output = open(path_energy, 'wb')
    pickle.dump(V_tensor(x_pred_list).cpu().detach().numpy(),output)
    output.close()

    output = open(path_force, 'wb')
    pickle.dump(np.sqrt(np.sum(g.cpu().detach().numpy()*g.cpu().detach().numpy(), axis=1)),output)
    output.close()

    output = open(path_cosloss, 'wb')
    pickle.dump(cos_batch,output)
    output.close()

    output = open(path_lg, 'wb')
    pickle.dump(lg_batch,output)
    output.close()

    output = open(path_lmax, 'wb')
    pickle.dump(lmax_batch,output)
    output.close()

    return

def train_pre(model):
    for i in range(10):
      a=np.array([-81.0640995,70.67714575])
      b=np.array([69.43047688,-67.71500979])

      c=(a+b)/2
      r=np.sqrt(np.sum((a-b)**2))/2

      optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

      t = np.random.uniform(0,1,(500,1))
      t = Variable(torch.from_numpy(t),requires_grad=True).to(device)

      xy=model(t)
      x,y=torch.split(xy,[1,1],dim=1)

      loss=torch.mean((x-c[0]-r*torch.cos(np.pi*t+2.398))**2+(y+c[1]+r*torch.sin(np.pi*t+2.398))**2)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      if i % 200==0:
        print(loss.item())
    torch.save(model.state_dict(), model_Parameters_name)

    return model



if __name__=='__main__':
    MEPpass_start_point=[-1.13366646,-0.03864257]
    MEPpass_end_point=[1.13366646,-0.03864257]


    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using {device} device to train")

    model = NeuralNetwork(MEPpass_start_point,MEPpass_end_point).to(device)
    model = model.double()

    model = train_pre(model)
    train(model)

